# Test some unsupervised methods

In [ ]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

### Start with fuzzy c means

In [ ]:

# Load your dataset with depth intervals (replace with your data)
data = pd.DataFrame({
    'log1': [/* your data */],
    'log2': [/* your data */],
    'log3': [/* your data */],
    'log4': [/* your data */]
})

# Scale the data after imputing just for the clustering step
scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')  # Temporarily impute for clustering only
data_scaled = scaler.fit_transform(imputer.fit_transform(data))

# Function for Fuzzy C-Means and calculating silhouette score
def fcm_and_silhouette(data, n_clusters):
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(data.T, n_clusters, 2, error=0.005, maxiter=1000, init=None)
    cluster_membership = np.argmax(u, axis=0)
    silhouette_avg = silhouette_score(data, cluster_membership)
    return silhouette_avg, cntr, u, cluster_membership

# Optimize the number of clusters
silhouette_scores = []
distortions = []
cluster_range = range(2, 10)

for n_clusters in cluster_range:
    silhouette_avg, cntr, u, cluster_membership = fcm_and_silhouette(data_scaled, n_clusters)
    silhouette_scores.append(silhouette_avg)
    distances = np.linalg.norm(data_scaled[:, np.newaxis] - cntr, axis=2) ** 2
    distortion = np.sum(np.min(distances, axis=1))
    distortions.append(distortion)

# Plot results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(cluster_range, silhouette_scores, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')

plt.subplot(1, 2, 2)
plt.plot(cluster_range, distortions, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Distortion')
plt.title('Elbow Method')
plt.tight_layout()
plt.show()

# Select the optimal number of clusters
optimal_clusters = cluster_range[np.argmax(silhouette_scores)]
print(f'Optimal number of clusters: {optimal_clusters}')

# Final FCM model with the optimal number of clusters
_, final_cntr, final_u, final_membership = fcm_and_silhouette(data_scaled, optimal_clusters)

# Restore original NaNs in the final DataFrame
data['Cluster'] = final_membership  # Add clusters to original data
print(data.head())